
# 💾 Databricks File System (DBFS)

## ¿Qué es DBFS?

**DBFS** (Databricks File System) es un **sistema de archivos distribuido** que actúa como una **capa de abstracción** para simplificar el almacenamiento y el acceso a los archivos dentro de un entorno de Databricks.

Su principal función es permitir que el almacenamiento en la nube (como S3, Azure Blob o ADLS) se trate como un sistema de archivos local, haciendo que el acceso a los datos sea mucho más fácil desde los notebooks y los clústeres.

***

## 🔑 Características Clave

DBFS proporciona varias ventajas fundamentales para el manejo de datos en Databricks:

* **Interfaz simple:** Ofrece una estructura de directorios y rutas estándar, facilitando la localización de los datos.
* **Almacenamiento persistente:** Los datos persisten incluso cuando el clúster de Spark se apaga.
* **Integración con clústeres Spark:** Permite que los procesos distribuidos de Spark accedan a los datos de manera eficiente.
* **Carpetas montadas:** Permite "montar" contenedores de almacenamiento en la nube bajo el prefijo `/mnt/`, simplificando el acceso y la seguridad.
* **Compatibilidad con múltiples formatos:** Soporta formatos comunes como Parquet, Delta Lake, CSV, JSON, etc.

***

## ⚙️ Acceso y Rutas

El acceso a DBFS se realiza mediante un prefijo especial y puede interactuar con distintos servicios:

### Prefijo de Ruta
Todas las rutas de DBFS comienzan con: `dbfs:/mnt/folder`

* **Ejemplo:** Para acceder a un archivo en una carpeta montada, se usaría una ruta como `dbfs:/mnt/mi_contenedor/datos.csv`.

### Integración con la Nube
DBFS se conecta directamente con los principales servicios de almacenamiento en la nube:
* **Azure:** Azure Blob Storage y Azure Data Lake Storage Gen2 (**ADLS Gen2**).
* **AWS:** Amazon S3 (**AWS S3**).

### Métodos de Acceso
Se puede interactuar con DBFS desde diferentes herramientas:
1.  **Lenguajes de Programación:** Python, Scala y R (usando `spark.read` o comandos del sistema de archivos).
2.  **Herramientas de Databricks:** Databricks CLI (Command Line Interface) y Databricks API.

# 🏛️ Arquitectura de Azure Databricks (Plano de Control y Plano de Cómputo)

Este diagrama ilustra cómo Azure Databricks se despliega dentro de la suscripción de Azure del cliente, separando las funciones de **Control** y **Cómputo**.

---

## 1. Plano de Control de Azure Databricks

Es el cerebro del entorno. Es administrado por Databricks y almacena metadatos y la configuración del workspace.

| Elemento | Función |
| :--- | :--- |
| **Databricks Web Application** | Interfaz principal para usuarios (autenticación SSO con Microsoft Entra ID). |
| **Notebooks y Jobs** | Permite gestionar, editar y ejecutar cuadernos (notebooks), así como la gestión de tareas recurrentes (`Jobs and query`). |
| **Cluster Management** | Gestiona la creación, escalado y terminación de los clústeres de Spark. |
| **REST API Client** | Permite la interacción programática con el workspace usando tokens revocables. |

***

## 2. Plano de Cómputo Clásico (VNet)

Aquí es donde ocurre el procesamiento real de los datos. Está alojado en una **Virtual Network (VNet)** dentro de la **suscripción de Azure del cliente**.

| Elemento | Función |
| :--- | :--- |
| **Compute Resources** | Incluye clústeres de Spark y **SQL Warehouses** (clásicos y *pro*) para el procesamiento de datos. |
| **Conexión Lógica** | Las flechas rojas punteadas indican peticiones lógicas (como iniciar clústeres o jobs) que **inician los recursos de cómputo** hacia el Plano de Control, garantizando la seguridad y que los recursos **no tengan direcciones IP públicas (No Public IP)**. |
| **Jobs Spark** | Ejecutan el código de los notebooks, generan *job logs* y almacenan los resultados. |

***

## 3. Almacenamiento de Datos y DBFS

El almacenamiento se divide en dos categorías principales, interactuando con DBFS.

### 📁 Workspace Root Storage (Raíz de DBFS)

* Almacena los **sistemas de archivos DBFS** (bibliotecas de archivos), los resultados de los jobs, los logs de los clústeres y las revisiones de los notebooks.
* Para los workspaces creados **después del 6 de marzo de 2023**, se usa **Azure Blob Storage**. Para los anteriores, se usaba ADLS Gen2.

### ☁️ Customer Data (Datos del Cliente)

* Datos externos y de negocio que se quieren procesar.
* Se almacenan típicamente en **ADLS Gen2** o **Blob Storage**.
* El plano de cómputo accede a estos datos directamente a través de **montajes DBFS** (`DBFS mount`) o acceso directo.

***

## 💡 El Papel de DBFS

**DBFS** (Databricks File System) proporciona la capa de abstracción.

* Permite a los clústeres de Spark acceder tanto al **Workspace Root Storage** como a los **Customer Data** mediante rutas unificadas, simplificando la lectura, escritura y el manejo de archivos.

### Flujo de Interacción

1.  El usuario/API interactúa con el **Plano de Control** (e.g., inicia un trabajo).
2.  El **Plano de Control** solicita al **Plano de Cómputo** (VNet) que inicie los recursos.
3.  El **Plano de Cómputo** (clústeres) procesa los datos, leyendo de los orígenes de datos (ADLS Gen2/Blob) a través de **DBFS** y enviando *logs* y resultados al **Plano de Control**.

# 💾 TRABAJANDO CON  Databricks File System (DBFS) ROOT 

In [0]:
%fs
ls dbfs:/

In [0]:
display(dbutils.fs.ls("/"))

In [0]:
display(dbutils.fs.ls("dbfs:/databricks-datasets/"))

#### Directorio rais del cluster

In [0]:
%sh
ls /

In [0]:
display(dbutils.fs.ls("/FileStore"))